In [1]:
from proto import *

In [2]:
with open('ontology_log_instance.ttl', 'r') as f:
    dat = np.array([i.strip().split(' ')[:3] for i in f.readlines()])
dat = filter_literals(dat)
np.random.shuffle(dat)

In [3]:
X_train = dat[:int(len(dat)*0.7)]
X_rest = dat[int(len(dat)*0.7):]

X, y = negative_sampling(X_rest, eta=1)
X_cal, X_test, y_cal, y_test = train_test_split(X, y)

X_cal, y_cal = filter_unseen(X_train, X_cal, y_cal)
X_test, y_test = filter_unseen(X_train, X_test, y_test)

print(X_cal[:5])
print(y_cal[:5])

[['<http://treat.net/onto.owl#alm_139591683_109>'
  '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>'
  '<http://treat.net/onto.owl#Alarm_Instance>']
 ['<http://treat.net/onto.owl#alarm_para_instance_2027>'
  '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>'
  '<http://treat.net/onto.owl#Alarm_Para_Instance>']
 ['<http://treat.net/onto.owl#alm_100001_16107>'
  '<http://treat.net/onto.owl#alarm_source>'
  '<http://treat.net/onto.owl#alarm_para_instance_1403>']
 ['<http://treat.net/onto.owl#alarm_para_instance_806>'
  '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>'
  '<http://treat.net/onto.owl#Pod_Instance>']
 ['<http://treat.net/onto.owl#alarm_para_instance_2125>'
  '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>'
  '<http://treat.net/onto.owl#alm_100058_15850>']]
[1. 1. 0. 1. 0.]


In [4]:
model = TransE(verbose=True)
model.fit(X_train)

Average TransE Loss:   1.104663: 100%|████████████████████████████████████████████| 100/100 [01:06<00:00,  1.49epoch/s]


In [5]:
calibrator = BetaCalibration()
probs = cal_and_eval(model, calibrator, X_cal, y_cal, X_test, y_test)
prob_triples = np.c_[(X_test, probs)]
df = pd.DataFrame(prob_triples,columns=['head', 'relation', 'tail', 'proba'])
df

MES: 0.12678772253468523
ECE: 0.0403563659066133


,head,relation,tail,proba
0,<http://treat.net/onto.owl#alm_100058_15796>,<http://treat.net/onto.owl#has_parameter>,<http://treat.net/onto.owl#alm_100001_164>,0.018527478
1,<http://treat.net/onto.owl#alm_143851523_349>,<http://treat.net/onto.owl#has_parameter>,<http://treat.net/onto.owl#alarm_para_instance...,0.83372027
2,<http://treat.net/onto.owl#alm_221257768_593>,<http://treat.net/onto.owl#has_parameter>,<http://treat.net/onto.owl#alm_5521_768>,0.032742742
3,<http://treat.net/onto.owl#alm_100058_15695>,<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>,<http://treat.net/onto.owl#Pod_Instance>,0.06448765
4,<http://treat.net/onto.owl#alarm_para_instance...,<http://treat.net/onto.owl#instance_of>,<http://treat.net/onto.owl#alm_para_182>,0.8684055
...,...,...,...,...
5233,<http://treat.net/onto.owl#alm_5521_614>,<http://treat.net/onto.owl#alarm_source>,<http://treat.net/onto.owl#alarm_para_instance_8>,0.8787009
5234,<http://treat.net/onto.owl#alm_100058_15635>,<http://treat.net/onto.owl#has_parameter>,<http://treat.net/onto.owl#alarm_para_instance...,0.8894671
5235,<http://treat.net/onto.owl#alm_100001_317>,<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>,<http://treat.net/onto.owl#alarm_para_instance...,6.1263337e-09
5236,<http://treat.net/onto.owl#alarm_para_instance...,<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>,<http://www.w3.org/2002/07/owl#NamedIndividual>,0.70281327


In [6]:
entities = set()
relations = set()
for x in X_train:
    h, r, t = x
    entities.add(h)
    relations.add(r)
    entities.add(t)

import random
from pprint import pprint
prob = 0
while prob < 0.5:
    h = random.sample(entities,1)[0]
    r = random.sample(relations, 1)[0]
    t = random.sample(entities, 1)[0]
    score = model.predict([h,r,t])
    prob = calibrator.transform(expit(score)).round(3)
    pprint([h, r, t])
    print(prob)


['<http://treat.net/onto.owl#alarm_para_instance_1783>',
 '<http://treat.net/onto.owl#host_cell>',
 '<http://treat.net/onto.owl#alm_100001_371>']
0.101
['<http://treat.net/onto.owl#alarm_para_instance_2299>',
 '<http://www.w3.org/2003/11/swrl#classPredicate>',
 '<http://treat.net/onto.owl#alm_80630>']
0.0
['<http://treat.net/onto.owl#alarm_para_instance_2288>',
 '<http://www.w3.org/2000/01/rdf-schema#subClassOf>',
 '<http://treat.net/onto.owl#alarm_para_instance_2536>']
0.455
['<http://treat.net/onto.owl#alarm_para_instance_770>',
 '<http://www.w3.org/2003/11/swrl#builtin>',
 '<http://treat.net/onto.owl#nfs_operation_44>']
0.0
['<http://treat.net/onto.owl#nfs_operation_37>',
 '<http://www.w3.org/2003/11/swrl#builtin>',
 '<http://treat.net/onto.owl#alarm_para_instance_3136>']
0.0
['<http://www.w3.org/2003/11/swrl#BuiltinAtom>',
 '<http://treat.net/onto.owl#depend_on>',
 '<http://treat.net/onto.owl#alm_100001_542>']
0.0
['<http://treat.net/onto.owl#alarm_para_instance_302>',
 '<http://tr

In [7]:
trps = []
for i in prob_triples:
    trps.append(Triple(i[0], i[1], i[2], float(i[3]), 10))
print(trps[:5])

[0.018527478::(<http://treat.net/onto.owl#alm_100058_15796>, <http://treat.net/onto.owl#has_parameter>, <http://treat.net/onto.owl#alm_100001_164>), 0.83372027::(<http://treat.net/onto.owl#alm_143851523_349>, <http://treat.net/onto.owl#has_parameter>, <http://treat.net/onto.owl#alarm_para_instance_1082>), 0.032742742000000005::(<http://treat.net/onto.owl#alm_221257768_593>, <http://treat.net/onto.owl#has_parameter>, <http://treat.net/onto.owl#alm_5521_768>), 0.06448765::(<http://treat.net/onto.owl#alm_100058_15695>, <http://www.w3.org/1999/02/22-rdf-syntax-ns#type>, <http://treat.net/onto.owl#Pod_Instance>), 0.8684055::(<http://treat.net/onto.owl#alarm_para_instance_2283>, <http://treat.net/onto.owl#instance_of>, <http://treat.net/onto.owl#alm_para_182>)]


In [11]:
from pprint import pprint
evds = make_synthetic_evidence(prob_triples)
pprint(evds[:5])

[array([0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1,
       1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1,
       0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1]),
 array([1, 0, 1, 1, 1, 1, 0, 1]),
 array([1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1,
       0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0]),
 array([1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0,
       0, 1, 0, 1, 1]),
 array([0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0,
       1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1,
       0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1])]


In [9]:
for i in range(len(trps)):
    trps[i].update_beta_a_b(evds[i])
print(trps[0])

0.06238288315789474::(<http://treat.net/onto.owl#alm_100058_15796>, <http://treat.net/onto.owl#has_parameter>, <http://treat.net/onto.owl#alm_100001_164>)


In [10]:
prob_triples2 = [t.to_list() for t in trps]
df2 = pd.DataFrame(prob_triples2,columns=['head', 'relation', 'tail', 'proba'])
df2

,head,relation,tail,proba
0,<http://treat.net/onto.owl#alm_100058_15796>,<http://treat.net/onto.owl#has_parameter>,<http://treat.net/onto.owl#alm_100001_164>,0.062383
1,<http://treat.net/onto.owl#alm_143851523_349>,<http://treat.net/onto.owl#has_parameter>,<http://treat.net/onto.owl#alarm_para_instance...,0.547108
2,<http://treat.net/onto.owl#alm_221257768_593>,<http://treat.net/onto.owl#has_parameter>,<http://treat.net/onto.owl#alm_5521_768>,0.193952
3,<http://treat.net/onto.owl#alm_100058_15695>,<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>,<http://treat.net/onto.owl#Pod_Instance>,0.460987
4,<http://treat.net/onto.owl#alarm_para_instance...,<http://treat.net/onto.owl#instance_of>,<http://treat.net/onto.owl#alm_para_182>,0.707362
...,...,...,...,...
5233,<http://treat.net/onto.owl#alm_5521_614>,<http://treat.net/onto.owl#alarm_source>,<http://treat.net/onto.owl#alarm_para_instance_8>,0.531390
5234,<http://treat.net/onto.owl#alm_100058_15635>,<http://treat.net/onto.owl#has_parameter>,<http://treat.net/onto.owl#alarm_para_instance...,0.549908
5235,<http://treat.net/onto.owl#alm_100001_317>,<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>,<http://treat.net/onto.owl#alarm_para_instance...,0.460674
5236,<http://treat.net/onto.owl#alarm_para_instance...,<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>,<http://www.w3.org/2002/07/owl#NamedIndividual>,0.629530
